In [1]:
# !pip3 install torch==1.2.0+cu92 torchvision==0.4.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html

In [2]:
import numpy as np
import pandas as pd


In [3]:
filepath = "/content/sample_data/1795_history.csv"
data = np.genfromtxt(filepath, delimiter='\t', dtype=str)[1:]

In [4]:
date   = data[:, 0]
open   = data[:, 1]
high   = data[:, 2]
low    = data[:, 3]
close  = data[:, 4]
volume = data[:, 5]
prices = np.array([close for date, open, high, low, close, volume in data]).astype(np.float64)
price = np.flip(prices)
price =pd.DataFrame(price, columns = ['Close'])


In [5]:
price.shape

(2453, 1)

In [6]:
def split_data(stock, lookback):
    data_raw = stock.to_numpy() # convert to numpy array
    data = []
    
    # create all possible sequences of length seq_len
    for index in range(len(data_raw) - lookback): 
        data.append(data_raw[index: index + lookback])
    
    data = np.array(data);
    test_set_size = int(np.round(0.2*data.shape[0]));
    train_set_size = data.shape[0] - (test_set_size);
    
    x_train = data[:train_set_size,:-1,:]
    y_train = data[:train_set_size,-1,:]
    
    x_test = data[train_set_size:,:-1]
    y_test = data[train_set_size:,-1,:]
    
    return [x_train, y_train, x_test, y_test]
lookback = 5 # choose sequence length
x_train, y_train, x_test, y_test = split_data(price, lookback)

In [7]:
# x_train

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim

In [9]:
x_train = torch.from_numpy(x_train).type(torch.Tensor)
x_test = torch.from_numpy(x_test).type(torch.Tensor)
y_train_lstm = torch.from_numpy(y_train).type(torch.Tensor)
y_test_lstm = torch.from_numpy(y_test).type(torch.Tensor)

In [10]:
# x_train

In [11]:
input_dim = 1
hidden_dim = 16
num_layers = 2
output_dim = 1

In [12]:
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers,dropout = 0.5, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :]) 
        return out

In [13]:
# class LSTM(nn.Module):
#     def __init__(self, lookback, num_layers=2, dropout=0.5, bidirectional=True):
#         super(LSTM, self).__init__()

#         # Nerual Layers
#         self.rnn   = nn.LSTM(lookback, 32, num_layers, dropout=dropout, bidirectional=True)
#         self.ly_a  = nn.Linear(32*(2 if bidirectional else 1), 16)
#         # self.ly_a  = nn.Linear(look_back, 16)
#         self.relu  = nn.ReLU()
#         self.reg   = nn.Linear(16, 1)

#     def predict(self, input):
#         with torch.no_grad():
#             return self.forward(input).item()

#     def forward(self, input):
#         r_out, (h_n, h_c) = self.rnn(input.unsqueeze(1), None)
#         # print(r_out.shape)
#         # input()
#         logits = self.reg(self.relu(self.ly_a(r_out.squeeze(1))))
#         # logits = self.reg(self.relu(self.ly_a(input)))

#         return logits

In [14]:
model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
criterion = torch.nn.MSELoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
import time
num_epochs =5000
hist = np.zeros(num_epochs)
start_time = time.time()
lstm = []

for t in range(num_epochs):
    y_train_pred = model(x_train)

    loss = criterion(y_train_pred, y_train_lstm)
    print("Epoch ", t, "MSE: ", loss.item())
    hist[t] = loss.item()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
training_time = time.time()-start_time
print("Training time: {}".format(training_time))

In [25]:
predict_data = [[[124.50],[125.506],[125.457],[124.173]]]
predict_data_arr = np.array(predict_data)
predict_data_arr_tensor = torch.tensor(predict_data_arr,dtype = torch.float32)
predict_data_arr_tensor
predict = model.forward(torch.tensor(predict_data_arr_tensor))
print('Predict Result', predict)

Predict Result tensor([[126.5784]], grad_fn=<AddmmBackward0>)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


In [17]:
# print(model)


In [18]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler(feature_range=(-1, 1))
# price['Close'] = scaler.fit_transform(price['Close'].values.reshape(-1,1))
# predict = pd.DataFrame(scaler.inverse_transform(y_train_pred.detach().numpy()))
# original = pd.DataFrame(scaler.inverse_transform(y_train_lstm.detach().numpy()))

In [19]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# sns.set_style("darkgrid")    

# fig = plt.figure()
# fig.subplots_adjust(hspace=0.2, wspace=0.2)

# plt.subplot(1, 2, 1)
# ax = sns.lineplot(x = original.index, y = original[0], label="Data", color='royalblue')
# ax = sns.lineplot(x = predict.index, y = predict[0], label="Training Prediction (LSTM)", color='tomato')
# ax.set_title('Stock price', size = 14, fontweight='bold')
# ax.set_xlabel("Days", size = 14)
# ax.set_ylabel("Cost (NTD)", size = 14)
# ax.set_xticklabels('', size=10)


# plt.subplot(1, 2, 2)
# ax = sns.lineplot(data=hist, color='royalblue')
# ax.set_xlabel("Epoch", size = 14)
# ax.set_ylabel("Loss", size = 14)
# ax.set_title("Training Loss", size = 14, fontweight='bold')
# fig.set_figheight(6)
# fig.set_figwidth(16)

In [20]:
# import math, time
# from sklearn.metrics import mean_squared_error

# # make predictions
# y_test_pred = model(x_test)

# # invert predictions
# y_train_pred = scaler.inverse_transform(y_train_pred.detach().numpy())
# y_train = scaler.inverse_transform(y_train_lstm.detach().numpy())
# y_test_pred = scaler.inverse_transform(y_test_pred.detach().numpy())
# y_test = scaler.inverse_transform(y_test_lstm.detach().numpy())

# # calculate root mean squared error
# trainScore = math.sqrt(mean_squared_error(y_train[:,0], y_train_pred[:,0]))
# print('Train Score: %.2f RMSE' % (trainScore))
# testScore = math.sqrt(mean_squared_error(y_test[:,0], y_test_pred[:,0]))
# print('Test Score: %.2f RMSE' % (testScore))
# lstm.append(trainScore)
# lstm.append(testScore)
# lstm.append(training_time)